<a href="https://colab.research.google.com/github/Javcm/Temas-selectos-de-Ciencia-de-Datos-Ayudantias-CIMAT/blob/main/Callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Callbacks

https://keras.io/api/callbacks/

* **Model checkpointing** : Guarda los pesos actuales del modelo en diferentes puntos del entrenamiento


* **Early stopping**: Interrumpe el entrenamiento cuando el error de validación no mejora (guarda el mejor modelo obtenido en el entrenamiento)

* **ReduceLROnPlateau**: Reduce el learning rate cuando el entrenamiento ha dejado de mejorar.




In [ ]:
import keras
keras.callbacks.ModelCheckpoint
keras.callbacks.EarlyStopping
keras.callbacks.ReduceLROnPlateau


keras.callbacks.CSVLogger

**THE MODELCHECKPOINT AND EARLYSTOPPING CALLBACKS**

* **EarlyStopping callback:** interrumpe el entrenamiento una vez que la metrica a monitorear deja de mejorar para un número fijo de epocas. Permite detener el entrenamiento antes de presentar problemas de overfitting


* **ModelCeckpoint**: permite guardr continuamente el modelo durante el entrenamiento (sólo guarda el mejor modelo: el modelo que registra la mejor actuación al final de cada epoca)

In [ ]:
import keras

#Los Callbacks se pasan como argumento en forma de lista en el entrenamiento (en el .fit)

callbacks_list = [keras.callbacks.EarlyStopping(monitor='acc', patience=10,), # Interrumpe entrenamiento cuando no hay mejora
                  
                  # monitor -> Parámetro a monitorear en la validación del modelo.
                  # patience -> Número a partir del cuál se interrumpe el entrenamiento cuando la métrica no mejora.
                  
                  keras.callbacks.ModelCheckpoint(filepath='my_model.h5', monitor='val_loss', save_best_only=True,)] # Saves the current weights after every epoch
                # filepath -> Dirección a guardar del modelo
                # monitor & save_best_only ->  Se guarda el modelo sólo si el error de validación mejora.        
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

# Note that because the callback will monitor validation loss and validation accuracy, you need to pass validation_data to the call to fit.
model.fit(X_train, Y_train, epochs=10, batch_size=32, callbacks=callbacks_list, validation_data=(X_test, Y_test))

**THE REDUCELRONPLATEAU CALLBACK**

* Reduce el learnig rate cuando el error de validación no mejora. Es una estrategia efectiva para salir de mínimos locales durante el entrenamiento

In [ ]:
callbacks_list = [
keras.callbacks.ReduceLROnPlateau(
monitor='val_loss', # Monitorea el error de validación del modelo
factor=0.1, # Divide la tasa de aprendizaje por 10 cuando se activa
patience=2, # El callback se activa a partir de que el error de validación deja de mejorar durante 2 epocas
)
]


model.fit(X_train, Y_train,
epochs=10,
batch_size=32,
callbacks=callbacks_list,
validation_data=(X_test, Y_test))

class BaseLogger: Callback that accumulates epoch averages of metrics.

class CSVLogger: Callback that streams epoch results to a CSV file.

class Callback: Abstract base class used to build new callbacks.

class CallbackList: Container abstracting a list of callbacks.

class EarlyStopping: Stop training when a monitored metric has stopped improving.

class History: Callback that records events into a History object.

class LambdaCallback: Callback for creating simple, custom callbacks on-the-fly.

class LearningRateScheduler: Learning rate scheduler.

class ModelCheckpoint: Callback to save the Keras model or model weights at some frequency.

class ProgbarLogger: Callback that prints metrics to stdout.

class ReduceLROnPlateau: Reduce learning rate when a metric has stopped improving.

class RemoteMonitor: Callback used to stream events to a server.

class TensorBoard: Enable visualizations for TensorBoard.

class TerminateOnNaN: Callback that terminates training when a NaN loss is encountered.

Ejemplo: Clasificación de Texto con TensorBoard

Datos -> IMDB data análisis de sentimientos

In [ ]:
import keras
from keras import layers
from keras.preprocessing import sequence
import numpy as np
from keras.datasets import imdb


max_features = 2000
max_len = 500

(X_train, Y_train), (X_test, Y_test) = imdb.load_data()
X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_test = sequence.pad_sequences(X_test, maxlen=max_len)

model = keras.models.Sequential()
model.add(layers.Embedding(max_features, 128,input_length=max_len,name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()
model.compile(optimizer='rmsprop',
loss='binary_crossentropy',
metrics=['acc'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
________________________________________________